In [3]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 14.0 MB/s eta 0:00:00


In [4]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# 1. Domain variabel
x_suhu = np.arange(0, 101, 1)
x_tekanan = np.arange(0, 11, 1)

# Fungsi keanggotaan Suhu
suhu_rendah = fuzz.trimf(x_suhu, [0, 0, 40])
suhu_sedang = fuzz.trimf(x_suhu, [30, 50, 70])
suhu_tinggi = fuzz.trimf(x_suhu, [60, 100, 100])

# Fungsi keanggotaan Tekanan
tekanan_rendah = fuzz.trimf(x_tekanan, [0, 0, 4])
tekanan_sedang = fuzz.trimf(x_tekanan, [3, 5, 7])
tekanan_tinggi = fuzz.trimf(x_tekanan, [6, 10, 10])

In [5]:
def sugeno(suhu_input, tekanan_input):
    # Fuzzifikasi
    u_s_rendah = fuzz.interp_membership(x_suhu, suhu_rendah, suhu_input)
    u_s_sedang = fuzz.interp_membership(x_suhu, suhu_sedang, suhu_input)
    u_s_tinggi = fuzz.interp_membership(x_suhu, suhu_tinggi, suhu_input)

    u_t_rendah = fuzz.interp_membership(x_tekanan, tekanan_rendah, tekanan_input)
    u_t_sedang = fuzz.interp_membership(x_tekanan, tekanan_sedang, tekanan_input)
    u_t_tinggi = fuzz.interp_membership(x_tekanan, tekanan_tinggi, tekanan_input)

    # Rule 1: suhu rendah AND tekanan rendah -> risiko = 10
    w1 = np.fmin(u_s_rendah, u_t_rendah)
    z1 = 10

    # Rule 2: suhu sedang AND tekanan sedang -> risiko = 50
    w2 = np.fmin(u_s_sedang, u_t_sedang)
    z2 = 50

    # Rule 3: suhu tinggi OR tekanan tinggi -> risiko = 90
    w3 = np.fmax(u_s_tinggi, u_t_tinggi)
    z3 = 90

    # Rule 4: suhu tinggi AND tekanan rendah -> risiko = 60
    w4 = np.fmin(u_s_tinggi, u_t_rendah)
    z4 = 60

    # Weighted average (Sugeno)
    w = np.array([w1, w2, w3, w4])
    z = np.array([z1, z2, z3, z4])

    # Menghitung hasil akhir (Z*)
    if np.sum(w) == 0:
        return 0
    result = np.sum(w * z) / np.sum(w)
    return result

In [6]:
def interactive_sugeno(suhu, tekanan):
    hasil = sugeno(suhu, tekanan)
    print(f"Hasil Prediksi Risiko (Sugeno): {hasil:.2f}")

    # Visualisasi
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Grafik Suhu
    ax1.plot(x_suhu, suhu_rendah, label="Rendah")
    ax1.plot(x_suhu, suhu_sedang, label="Sedang")
    ax1.plot(x_suhu, suhu_tinggi, label="Tinggi")
    ax1.axvline(suhu, color='red', linestyle='--')
    ax1.set_title("Fungsi Keanggotaan Suhu")
    ax1.legend()

    # Grafik Tekanan
    ax2.plot(x_tekanan, tekanan_rendah, label="Rendah")
    ax2.plot(x_tekanan, tekanan_sedang, label="Sedang")
    ax2.plot(x_tekanan, tekanan_tinggi, label="Tinggi")
    ax2.axvline(tekanan, color='red', linestyle='--')
    ax2.set_title("Fungsi Keanggotaan Tekanan")
    ax2.legend()

    plt.show()

# Menjalankan slider
interact(
    interactive_sugeno,
    suhu=FloatSlider(min=0, max=100, step=1, value=50),
    tekanan=FloatSlider(min=0, max=10, step=0.5, value=5)
)

interactive(children=(FloatSlider(value=50.0, description='suhu', step=1.0), FloatSlider(value=5.0, descriptio…

<function __main__.interactive_sugeno(suhu, tekanan)>